# Introduction
To unpack the ZIP-file with JSON-files at least two approach are possible. They are depends from
1. Unpack the ZIP-file on storage and read the files into pandas Data Frame. The reading of files is possible in parallel
2. To save the storage it is possible to read the ZIP-file on-the-fly into pandas Data Frame without to unpack them on the storage.

In the test task I decided to use the approach Nr.2 to safe the storage.

To store the data for the future Analysis (not include in this task) the good approach is to save the pandas Data Frame to Delta tables and not as CSV file. Delta tables have optimized  storage format and provides optimized performance for analytics workloads

# Cleaning of data

1. The columns *group_vehicle_number* and *total_driven_km* should not have the NULL values
2. The column *total_driven_km* should have the positive data
3. The dupplicates would be find out by the column *pk_hash* (all values from the file)

# More cleaning of data

1. The column *record_country* should have only valid values (possible check with **pycountry** package)
2. The column *record_date* should have th valid date. UTC?
3. The **group_vehicle_number** should not have the special charachters

**Please set the path to your ZIP-file first**

In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
data_zip_file = '/content/drive/MyDrive/Colab Notebooks/car/data.zip'

**Extract ZIP file to storage --> DISABLED**

In [106]:
import os
import zipfile


# Specify the extract dir (s. mount path above)
# create the target directory if it doesn't exist
# and unzip it

# data_extract_dir = '/content/drive/MyDrive/Colab Notebooks/car'
# os.makedirs(data_extract_dir, exist_ok=True)

#with zipfile.ZipFile(data_zip_file, 'r') as zip_ref:
#    zip_ref.extractall(data_extract_dir)

**Reading ZIP file without extracting them to storage**

In [107]:
import zipfile
import pandas as pd
import json
import io
import hashlib

# Create an empty list to store dataframes
# Open the zip file in read mode
# Iterate over each file in the zip archive
# Assumed: in the zip archive are only '.json' files
# Open the file inside the zip archive as a stream
# Load JSON data directly from the stream into a pandas dataframe
# Calculate row-wise MD5 hash using all column values and add as a new column 'pk_hash' to find a duplicates
# Append the dataframe to the list of dataframes
# Concatenate all dataframes into a single dataframe

dfs = []
count_files = 0
error_files = 0

with zipfile.ZipFile(data_zip_file, 'r') as zip_ref:

    for file_name in zip_ref.namelist():
      count_files += 1
      with zip_ref.open(file_name) as file:
          with io.TextIOWrapper(file, encoding='utf-8') as text_file:
            try:
              df = pd.DataFrame([json.loads(line) for line in text_file])
              df['pk_hash'] = df.apply(lambda row: hashlib.sha256(','.join(map(str, row)).encode('utf-8')).hexdigest(), axis=1)
              df['file_name'] = file_name
              dfs.append(df)
            except Exception as error:
              error_files += 1
              print(f"Error reading file: {file_name}, Error: {error}")


combined_df = pd.concat(dfs, ignore_index=True)

print("\n")
print("Step: start of UNZIPPING OF DELIVERY")
print(f"Count of files total: {count_files}, files with errors total: {error_files}")
print("Step: end of UNZIPPING OF DELIVERY")
print("\n")


Error reading file: data/gmdm_2412.json, Error: Invalid \uXXXX escape: line 1 column 133 (char 132)
Error reading file: data/gmdm_3200.json, Error: Invalid \uXXXX escape: line 1 column 133 (char 132)


Step: start of UNZIPPING OF DELIVERY
Count of files total: 3321, files with errors total: 2
Step: end of UNZIPPING OF DELIVERY




In [108]:
combined_df.head()

,pk_hash,file_name,group_vehicle_number,record_country,record_date,comment,total_driven_km
0,9acf564f12bb136559b2ed2d7382b45a6ae60d13da56b5...,data/gmdm_0.json,5h801818528,AT,2022-09-13T12:44:44.458+01:00,HIMMEL_WOLKENLOS,949
1,eabc40e3fbae750c98d9c615e349196157c803915f7151...,data/gmdm_1.json,8ö542403911,AU,2023-12-09T13:09:18.454+13:00,COMMENT,957
2,4fcd2af38a4e857c6b3bd64ba3d0ecfb192a9a4fbd3d4b...,data/gmdm_10.json,ui654338264,AU,2023-12-10T14:18:17.045+10:30,COMMENT,154
3,45a583d9d931c396ace7056a8e86f67c6dd3c6fc12336d...,data/gmdm_100.json,#h746322144,DE,2024-03-06T22:51:47.722+01:00,COMMENT,324
4,68ba7a6afea10435ab2c243795600bfebf583eebe0ca9c...,data/gmdm_1000.json,03659746900,IT,2021-12-15T17:21:44.031+01:00,COMMENT,684


In [109]:
combined_df.tail()

,pk_hash,file_name,group_vehicle_number,record_country,record_date,comment,total_driven_km
3314,ac6ed2b997deb01fbe9886a071383bb3ab1af55b010cf7...,data/gmdm_995.json,uw452295905,DE,2021-05-31T08:48:19.000Z,NaN,477
3315,c25e1e94684ca742f114a386cb90543d160880a8cba061...,data/gmdm_996.json,fL431831576,DE,2022-09-06T13:56:44.002+02:00,Systemstatus,632
3316,3ecd6826eae0a558605bddd04989b8d6c1bd586e347ac5...,data/gmdm_997.json,#T173924603,DE,2023-06-02T14:03:18.844+02:00,NaN,390
3317,94ba70929794efc7cc7f610da438b0e536380b9b2e377d...,data/gmdm_998.json,sO103810932,DE,2021-09-30T15:04:02.495+02:00,Systemstatus,173
3318,ecaee9ceb8b8d9dcf01becad65d0863875e7982e566ddd...,data/gmdm_999.json,qn751836261,DE,2022-03-23T14:34:01.081+01:00,COMMENT,528


In [110]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3319 entries, 0 to 3318
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   pk_hash               3319 non-null   object
 1   file_name             3319 non-null   object
 2   group_vehicle_number  3304 non-null   object
 3   record_country        3317 non-null   object
 4   record_date           3317 non-null   object
 5   comment               1891 non-null   object
 6   total_driven_km       3316 non-null   object
dtypes: object(7)
memory usage: 181.6+ KB


In [111]:
combined_df.describe(include='all')

,pk_hash,file_name,group_vehicle_number,record_country,record_date,comment,total_driven_km
count,3319,3319,3304,3317,3317,1891,3316
unique,3317,3318,3265,33,3313,118,973
top,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,data/gmdm_2634.json,,DE,2022-09-01T12:55:16.217Z,COMMENT,405
freq,2,2,38,2293,2,831,9


In [112]:
# Remove rows where 'group_vehicle_number' and 'total_driven_km are NaN
import ast

columns_to_check = "'group_vehicle_number', 'total_driven_km'"
column_list = ast.literal_eval(f"[{columns_to_check}]")

group_vehicle_number_count = combined_df[column_list].isna().sum()
print("Count NaN in 'group_vehicle_number' and 'total_driven_km':", group_vehicle_number_count)

group_vehicle_number_total_driven_km_df_cleaned = combined_df.dropna(subset=column_list)

Count NaN in 'group_vehicle_number' and 'total_driven_km': group_vehicle_number    15
total_driven_km          3
dtype: int64


In [113]:
group_vehicle_number_total_driven_km_df_cleaned.describe()

,pk_hash,file_name,group_vehicle_number,record_country,record_date,comment,total_driven_km
count,3303,3303,3303,3303,3303,1891,3303
unique,3302,3302,3265,33,3300,118,973
top,f09c53c840cb424a9138961d4a495d8dcb5ac2742da765...,data/gmdm_2634.json,,DE,2021-06-02T09:14:03.000Z,COMMENT,405
freq,2,2,38,2289,2,831,9


In [114]:
# Remove negative 'total_driven_km' and convert it in int
group_vehicle_number_total_driven_km_df_cleaned['total_driven_km'] = pd.to_numeric(group_vehicle_number_total_driven_km_df_cleaned['total_driven_km'], errors='coerce')

total_driven_km_df_cleaned = group_vehicle_number_total_driven_km_df_cleaned[group_vehicle_number_total_driven_km_df_cleaned['total_driven_km'] >= 0]


<ipython-input-114-d9e1c81aaf38>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_vehicle_number_total_driven_km_df_cleaned['total_driven_km'] = pd.to_numeric(group_vehicle_number_total_driven_km_df_cleaned['total_driven_km'], errors='coerce')


In [115]:
total_driven_km_df_cleaned.describe(include='all')

,pk_hash,file_name,group_vehicle_number,record_country,record_date,comment,total_driven_km
count,3302,3302,3302,3302,3302,1890,3.302000e+03
unique,3301,3301,3264,33,3299,118,NaN
top,f09c53c840cb424a9138961d4a495d8dcb5ac2742da765...,data/gmdm_2634.json,,DE,2021-06-02T09:14:03.000Z,COMMENT,NaN
freq,2,2,38,2288,2,831,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,5.896166e+08
std,NaN,NaN,NaN,NaN,NaN,NaN,3.388113e+10
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.480000e+02
50%,NaN,NaN,NaN,NaN,NaN,NaN,4.910000e+02
75%,NaN,NaN,NaN,NaN,NaN,NaN,7.490000e+02


In [116]:
# Identify rows with duplicate pk_hash values
# duplicate_pk_hashes = data_df_cleaned[data_df_cleaned.duplicated(subset='pk_hash', keep=False)]

# Display rows with duplicate pk_hash values
# print("Rows with Duplicate pk_hash Values:")
# print(duplicate_pk_hashes)

# Drop duplicates pk_hash after group_vehicle_number was cleaned
pk_hash_df_cleaned = total_driven_km_df_cleaned.drop_duplicates(subset='pk_hash', keep='first')

In [117]:
pk_hash_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3301 entries, 0 to 3318
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pk_hash               3301 non-null   object 
 1   file_name             3301 non-null   object 
 2   group_vehicle_number  3301 non-null   object 
 3   record_country        3301 non-null   object 
 4   record_date           3301 non-null   object 
 5   comment               1889 non-null   object 
 6   total_driven_km       3301 non-null   float64
dtypes: float64(1), object(6)
memory usage: 206.3+ KB


In [118]:
pk_hash_df_cleaned.describe(include='all')

,pk_hash,file_name,group_vehicle_number,record_country,record_date,comment,total_driven_km
count,3301,3301,3301,3301,3301,1889,3.301000e+03
unique,3301,3301,3264,33,3299,118,NaN
top,9acf564f12bb136559b2ed2d7382b45a6ae60d13da56b5...,data/gmdm_0.json,,DE,2021-06-02T09:14:03.000Z,COMMENT,NaN
freq,1,1,38,2287,2,831,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,5.897952e+08
std,NaN,NaN,NaN,NaN,NaN,NaN,3.388626e+10
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.480000e+02
50%,NaN,NaN,NaN,NaN,NaN,NaN,4.910000e+02
75%,NaN,NaN,NaN,NaN,NaN,NaN,7.490000e+02
